In [1]:
from conversion_gridtool import *

In [2]:
# Set neighbourhood threshold radius to determine, how close endnodes have to be together to get grouped
neighbourhood_threshold = 100

# Max. length of a line which can be a type 'busbar', in km
busbar_max_length = 1

# Multiplier factor for the exported length of line (slack compensation)
way_length_multiplier = 1.2

In [3]:
# # Define boolean options
# bool_options = {
#     'calculate_real_line_length': True,  # 是否计算实际线路长度
#     'beeline_visu_threshold_diff_percent': 5,  # 视觉化时的阈值（百分比）
#     'beeline_visu_threshold_diff_absolut': 0.5,  # 视觉化时的阈值（绝对值，公里）

#     # 推荐的可视化选项
#     'plot_ways_original': True,  # 可视化所有选定的线路（原始数据集）
#     'plot_ways_grouping': False,  # 在分组时可视化所有选定的线路
#     'plot_ways_final': True,  # 在地图上可视化所有选定的线路（最终数据集）
#     'histogram_distances_between_endpoints': False,  # 可视化所有端点之间的距离
#     'plot_comparison_real_beeline': False,  # 比较实际线路和直线的可视化

#     # 可选的可视化选项（用于调试和深入研究）
#     'histogram_length_busbars': False,  # 可视化巴士杆的长度
#     'histogram_stacked_endnodes': False,  # 可视化堆叠的端点数量
#     'plot_stacked_endnodes': False,  # 在地图上可视化所有堆叠的端点
#     'histogram_neighbouring_endnodes': False,  # 可视化相邻的端点数量
#     'plot_neighbouring_endnodes': False,  # 在地图上可视化所有相邻的端点
# }

In [4]:
gdf = gpd.read_file("power_data/vietnam-latest_lines.gpkg")

In [5]:
# Specify the path to the data file (you can modify this path as needed)
file_path = '../inputs/2024-08-20_Vietnam_110kV_to_500kV.json'
data_raw = import_json(file_path)

Start importing Data (*.json file)...
   ... finished!


In [6]:
with open('../inputs/2024-08-20_Vietnam_110kV_to_500kV.json') as f:
    data = json.load(f)
    
unique_types = set([element['type'] for element in data['elements']])
print(unique_types)
print(len(data['elements']))

{'node', 'way', 'relation'}
82421


In [7]:
# Separate 'node' and 'way' elements and add UID
data_nodes_all, data_ways_all = separate_raw_data(data_raw)

Start separating raw data into way- and node-elements... (takes a few seconds)
   ... finished! (0.052 seconds) 



In [8]:
for element in data['elements']:
    if element['type'] == 'relation':
        print(element)

{'type': 'relation', 'id': 13678398, 'members': [{'type': 'way', 'ref': 884244627, 'role': ''}, {'type': 'way', 'ref': 1050633306, 'role': ''}, {'type': 'way', 'ref': 1050633307, 'role': ''}, {'type': 'way', 'ref': 1091498926, 'role': ''}, {'type': 'way', 'ref': 1091498924, 'role': ''}, {'type': 'way', 'ref': 1091498923, 'role': ''}, {'type': 'way', 'ref': 1050633308, 'role': ''}, {'type': 'way', 'ref': 1091498921, 'role': ''}, {'type': 'way', 'ref': 1091498920, 'role': ''}, {'type': 'way', 'ref': 1247577282, 'role': ''}, {'type': 'way', 'ref': 1050633309, 'role': ''}, {'type': 'way', 'ref': 1050633310, 'role': ''}, {'type': 'way', 'ref': 1050633305, 'role': ''}, {'type': 'way', 'ref': 1091498908, 'role': ''}, {'type': 'way', 'ref': 1091498907, 'role': ''}, {'type': 'way', 'ref': 1091498904, 'role': ''}, {'type': 'way', 'ref': 1091498938, 'role': ''}, {'type': 'way', 'ref': 1091498937, 'role': ''}, {'type': 'way', 'ref': 1091498936, 'role': ''}, {'type': 'way', 'ref': 1091498913, 'role

In [9]:
len(data_nodes_all)

76720

In [10]:
len(data_ways_all)

5698

In [11]:
# Add coordinates and lengths to ways
data_ways_all_gdf = add_coordinates_length(data_ways_all, data_nodes_all)

Start adding coordinates to each way...


Finished adding coordinates!


In [12]:
# for col in data_ways_all.columns:
#     if data_ways_all[col].apply(lambda x: isinstance(x, list)).any():
#         print(data_ways_all[col])
#         # 将列表列转换为字符串，或者你可以选择将该列删除
#         data_ways_all[col] = data_ways_all[col].apply(lambda x: str(x) if isinstance(x, list) else x)

In [13]:
# data_ways_all.to_file("../outputs/data_ways_all_after_add_coordinates.gpkg", layer='data_ways_all', driver="GPKG", crs='32648')

In [14]:
# import geopandas as gpd
# import pandas as pd
# import json
# from shapely.geometry import LineString

# # 读取 JSON 文件
# with open('../inputs/2024-08-20_Vietnam_110kV_to_500kV.json') as f:
#     data = json.load(f)

# # 初始化列表来存储数据
# ids = []
# geometries = []
# tags = []

# unique_types = set([element['type'] for element in data['elements']])
# print(unique_types)

# # 提取每条线路数据
# for element in data['elements']:
#     if element['type'] == 'way':
#         # 提取线路 ID 和节点 ID 列表
#         line_id = element['id']
#         nodes = element['nodes']
        
#         # 将节点 ID 列表转换为 LineString (示例假设你有坐标数据)
#         # 注意：如果没有坐标数据，需要从节点 ID 映射到实际坐标，这里假设已处理
#         # points = [mapping[node_id] for node_id in nodes] # 示例
#         # line = LineString(points)
        
#         # 示例代码中用节点 ID 直接生成 LineString
#         line = LineString([(node_id, node_id) for node_id in nodes])  # 假数据

#         # 存储线路 ID 和几何数据
#         ids.append(line_id)
#         geometries.append(line)
#         tags.append(element.get('tags', {}))  # 提取 tags 数据

# # 创建 GeoDataFrame
# gdf = gpd.GeoDataFrame({
#     'id': ids,
#     'geometry': geometries,
#     'tags': tags  # 你也可以分离 tags 内部字段
# }, geometry='geometry', crs="EPSG:4326")

# gdf = gdf.to_crs(epsg=32648)

# # 保存为 GeoPackage 文件
# gdf.to_file("../outputs/2024-08-20_Vietnam_110kV_to_500kV.gpkg", layer='lines', driver="GPKG")


In [15]:
data_ways_all_columns = expand_tags_to_column(data_ways_all_gdf)

Unique keys found in tags: {'to', 'voltage', 'fixme', 'frequency', 'power', 'design', 'wires', 'not:operator:wikidata', 'distance', 'ref', 'note', 'construction', 'circuits', 'operator', 'route', 'line', 'from', 'operator:wikipedia', 'name', 'location', 'cables', 'layer', 'operator:wikidata', 'source', 'type', 'alt_name', 'name:en'}


In [16]:
# data_ways_all_update = replace_empty_with_nan(data_ways_all_update)

In [17]:
data_ways_all_update_voltage = count_voltage_levels(data_ways_all_columns)

Start counting voltage levels...


Voltage levels count:
Voltage level 110 kV: 2022
Voltage level 220 kV: 951
Voltage level 500 kV: 2587
Count of rows with None voltage: 187


In [18]:
len(data_ways_all_update_voltage.columns.tolist())

32

In [19]:
print(data_ways_all_update_voltage['voltage'].dtype)

Int64


In [20]:
data_ways_all_update_voltage['Length'].dtype

dtype('float64')

In [21]:
# Find all ways with type busbars, extract them and delete them
data_ways_selected, data_busbars = delete_busbars(data_ways_all_update_voltage, busbar_max_length=1)

Start deleting ways with type "busbar" or "bay"...
   ATTENTION! Way Element ID 1293378775 has type "busbar" or "bay", but is too long.
               Length: 31.64 km of max. 1.0 km
               This way won't be added to the "busbar" exception list.
   ... there are 7 busbars/bays in total
   ... 6 busbars have been deleted
   ... finished! (0.197 seconds) 



In [22]:
# Detect all possible DC lines
data_ways_selected, dc_candidates = count_possible_dc(data_ways_selected)

Start detecting lines which could be DC lines...


ValueError: invalid literal for int() with base 10: ''

In [23]:
dc_candidates

NameError: name 'dc_candidates' is not defined

In [25]:
data_ways_selected

,type,id,nodes,geometry,distance_m,alt_name,voltage,frequency,construction,distance,...,wires,operator,line,ref,fixme,from,name,vlevels,busbar,dc_candidate
0,NaN,153608701,"[11762056230, 1662610703, 1662610705, 16626107...",LINESTRING (610714.8526952094 2315180.69968158...,2243.899524,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False
1,NaN,176708144,"[1871844780, 1871844783, 11068562853, 18718448...",LINESTRING (672622.408581327 1278800.027259281...,22806.373190,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False
2,NaN,241194661,"[5311792555, 5311792557, 5311792558, 531179255...",LINESTRING (682154.0170507935 2315887.78970822...,13446.478256,NaN,220000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False
3,NaN,241255651,"[2489447617, 2489447612, 2489447624, 248944762...",LINESTRING (684889.726446027 2304273.826937457...,2177.637585,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False
4,NaN,241255652,"[2489447614, 2489447607, 2489447605, 248944760...",LINESTRING (684710.9143918614 2304282.78396018...,18356.684673,NaN,220000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,NaN,1096518667,"[10041470211, 10041470213, 10041470214]",LINESTRING (759612.8600952344 1823344.26089980...,609.589101,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False
5743,NaN,1096518668,"[10041470214, 10041470215, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1027.988981,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False
5744,NaN,1096518669,"[10041470214, 10041470217, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1028.388216,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False
5745,NaN,1096518679,"[10041470241, 10041470243, 10041470244]",LINESTRING (766949.0038125969 1815082.46899628...,665.572341,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False


In [26]:
# Count how many cables a way has (needed to double or triple a way), add flags
data_ways_selected = count_cables(data_ways_selected)

Start counting cables per way...
   ATTENTION! Unknown cable number ("nan") in ID 176708144. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 241255651. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 307777845. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400539088. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400539090. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400552552. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400552553. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 400564898. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 430731461. This way wont be cloned automatically.
   ATTENTION! Unknown cable number ("nan") in ID 468821784. This way wont be

In [27]:
data_ways_selected

,type,id,nodes,geometry,distance_m,alt_name,voltage,frequency,construction,distance,...,operator,line,ref,fixme,from,name,vlevels,busbar,dc_candidate,systems
0,NaN,153608701,"[11762056230, 1662610703, 1662610705, 16626107...",LINESTRING (610714.8526952094 2315180.69968158...,2243.899524,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False,2.0
1,NaN,176708144,"[1871844780, 1871844783, 11068562853, 18718448...",LINESTRING (672622.408581327 1278800.027259281...,22806.373190,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False,NaN
2,NaN,241194661,"[5311792555, 5311792557, 5311792558, 531179255...",LINESTRING (682154.0170507935 2315887.78970822...,13446.478256,NaN,220000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False,2.0
3,NaN,241255651,"[2489447617, 2489447612, 2489447624, 248944762...",LINESTRING (684889.726446027 2304273.826937457...,2177.637585,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False,NaN
4,NaN,241255652,"[2489447614, 2489447607, 2489447605, 248944760...",LINESTRING (684710.9143918614 2304282.78396018...,18356.684673,NaN,220000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,False,False,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,NaN,1096518667,"[10041470211, 10041470213, 10041470214]",LINESTRING (759612.8600952344 1823344.26089980...,609.589101,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False,NaN
5743,NaN,1096518668,"[10041470214, 10041470215, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1027.988981,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False,NaN
5744,NaN,1096518669,"[10041470214, 10041470217, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1028.388216,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False,NaN
5745,NaN,1096518679,"[10041470241, 10041470243, 10041470244]",LINESTRING (766949.0038125969 1815082.46899628...,665.572341,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,False,False,NaN


In [28]:
# Calculate distances between all endpoints
distances_between_nodes = my_calc_distances_between_endpoints(data_ways_selected, plot_histogram=True)

In [29]:
np.min(distances_between_nodes[distances_between_nodes > 0])

0.16339071643766542

In [30]:
# Calculate all stacked nodes
data_ways_selected, nodes_stacked_pairs = my_calc_stacked_endnodes(data_ways_selected, distances_between_nodes, bool_options)

Start finding all stacked endnodes...
... 8622 endnodes are stacked!
... finished! (0.502 seconds)



/scistor/ivm/mye500/projects/cascading_failure/power_flow_model/simplify_old.py:666: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i_ways, 'node1_stacked'] = False
/scistor/ivm/mye500/projects/cascading_failure/power_flow_model/simplify_old.py:667: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'True' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  data.at[i_ways, 'node2_stacked'] = True


In [31]:
# Calculate neighbouring nodes
data_ways_selected, nodes_neighbouring_pairs = my_calc_neighbouring_endnodes(data_ways_selected, distances_between_nodes, neighbourhood_threshold, bool_options)

Start finding all neighbouring endnodes...
... 5547 endnodes are in the same neighbourhood!
... finished!


In [32]:
# Group stacked nodes
nodes_stacked_grouped = my_group_nodes(nodes_stacked_pairs)

Start grouping all pairs from "pairs_input" (may take a few seconds)...
   ... 8622 nodes will be grouped together in 2745 groups,
       with an average of 3.14 nodes per group.
   ... finished! (1.999 seconds)



In [33]:
# Group neighbouring nodes                               
nodes_neighbouring_grouped = my_group_nodes(nodes_neighbouring_pairs)

Start grouping all pairs from "pairs_input" (may take a few seconds)...
   ... 5547 nodes will be grouped together in 894 groups,
       with an average of 6.20 nodes per group.
   ... finished! (0.924 seconds)



In [34]:
# Add coordinates of stacked endnodes
data_ways_selected = group_stacked_endnodes(data_ways_selected, nodes_stacked_grouped)

Completed updating coordinates for stacked groups.


In [35]:
# Add coordinates of neighbouring endnodes
data_ways_selected = group_stacked_endnodes(data_ways_selected, nodes_neighbouring_grouped)

Completed updating coordinates for stacked groups.


In [36]:
data_ways_selected

,type,id,nodes,geometry,distance_m,alt_name,voltage,frequency,construction,distance,...,node1_stacked,node2_stacked,node1_neighbour,node2_neighbour,ID_node1_grouped,ID_node2_grouped,lon1_grouped,lat1_grouped,lon2_grouped,lat2_grouped
0,NaN,153608701,"[11762056230, 1662610703, 1662610705, 16626107...",LINESTRING (610714.8526952094 2315180.69968158...,2243.899524,NaN,110000,50,NaN,NaN,...,False,True,False,False,1.176206e+10,NaN,610714.852695,2.315181e+06,NaN,NaN
1,NaN,176708144,"[1871844780, 1871844783, 11068562853, 18718448...",LINESTRING (672622.408581327 1278800.027259281...,22806.373190,NaN,110000,50,NaN,NaN,...,False,True,False,False,NaN,1.111322e+10,NaN,NaN,651889.179899,1.285444e+06
2,NaN,241194661,"[5311792555, 5311792557, 5311792558, 531179255...",LINESTRING (682154.0170507935 2315887.78970822...,13446.478256,NaN,220000,50,NaN,NaN,...,False,True,False,False,1.111322e+10,2.489448e+09,651889.179899,1.285444e+06,684741.069151,2.304331e+06
3,NaN,241255651,"[2489447617, 2489447612, 2489447624, 248944762...",LINESTRING (684889.726446027 2304273.826937457...,2177.637585,NaN,110000,50,NaN,NaN,...,True,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,241255652,"[2489447614, 2489447607, 2489447605, 248944760...",LINESTRING (684710.9143918614 2304282.78396018...,18356.684673,NaN,220000,50,NaN,NaN,...,True,True,False,False,NaN,5.311793e+09,NaN,NaN,672301.251944,2.304724e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,NaN,1096518667,"[10041470211, 10041470213, 10041470214]",LINESTRING (759612.8600952344 1823344.26089980...,609.589101,NaN,<NA>,NaN,NaN,NaN,...,True,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN
5743,NaN,1096518668,"[10041470214, 10041470215, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1027.988981,NaN,<NA>,NaN,NaN,NaN,...,True,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN
5744,NaN,1096518669,"[10041470214, 10041470217, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1028.388216,NaN,<NA>,NaN,NaN,NaN,...,True,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN
5745,NaN,1096518679,"[10041470241, 10041470243, 10041470244]",LINESTRING (766949.0038125969 1815082.46899628...,665.572341,NaN,<NA>,NaN,NaN,NaN,...,True,True,False,False,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Add final coordinates, hence select from original or grouped coordinates                          
data_ways_selected = my_add_final_coordinates(data_ways_selected)

Start adding final coordinates...
... finished!


In [38]:
#------------------------------------------------

In [39]:
data_ways_selected, data_singular_ways = my_delete_singular_ways(data_ways_selected)

Start deleting ways which have the same endpoints after grouping...
   ... 1225 ways were deleted!
   ... finished! (0.006 seconds)


In [40]:
len(data_singular_ways)

1225

In [41]:
data_ways_selected

,type,id,nodes,geometry,distance_m,alt_name,voltage,frequency,construction,distance,...,lon1_grouped,lat1_grouped,lon2_grouped,lat2_grouped,ID_node1_final,lon1_final,lat1_final,ID_node2_final,lon2_final,lat2_final
0,NaN,153608701,"[11762056230, 1662610703, 1662610705, 16626107...",LINESTRING (610714.8526952094 2315180.69968158...,2243.899524,NaN,110000,50,NaN,NaN,...,610714.852695,2.315181e+06,NaN,NaN,1.176206e+10,610714.852695,2.315181e+06,1.176206e+10,609979.512105,2.317122e+06
1,NaN,176708144,"[1871844780, 1871844783, 11068562853, 18718448...",LINESTRING (672622.408581327 1278800.027259281...,22806.373190,NaN,110000,50,NaN,NaN,...,NaN,NaN,651889.179899,1.285444e+06,1.871845e+09,672622.408581,1.278800e+06,1.111322e+10,651889.179899,1.285444e+06
2,NaN,241194661,"[5311792555, 5311792557, 5311792558, 531179255...",LINESTRING (682154.0170507935 2315887.78970822...,13446.478256,NaN,220000,50,NaN,NaN,...,651889.179899,1.285444e+06,684741.069151,2.304331e+06,1.111322e+10,651889.179899,1.285444e+06,2.489448e+09,684741.069151,2.304331e+06
3,NaN,241255651,"[2489447617, 2489447612, 2489447624, 248944762...",LINESTRING (684889.726446027 2304273.826937457...,2177.637585,NaN,110000,50,NaN,NaN,...,NaN,NaN,NaN,NaN,2.489448e+09,684889.726446,2.304274e+06,2.489448e+09,683277.940869,2.305527e+06
4,NaN,241255652,"[2489447614, 2489447607, 2489447605, 248944760...",LINESTRING (684710.9143918614 2304282.78396018...,18356.684673,NaN,220000,50,NaN,NaN,...,NaN,NaN,672301.251944,2.304724e+06,2.489448e+09,684710.914392,2.304283e+06,5.311793e+09,672301.251944,2.304724e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4517,NaN,1096518667,"[10041470211, 10041470213, 10041470214]",LINESTRING (759612.8600952344 1823344.26089980...,609.589101,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.004147e+10,759612.860095,1.823344e+06,1.004147e+10,760168.657366,1.823133e+06
4518,NaN,1096518668,"[10041470214, 10041470215, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1027.988981,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.004147e+10,760168.657366,1.823133e+06,1.004147e+10,760939.202088,1.822452e+06
4519,NaN,1096518669,"[10041470214, 10041470217, 10041470216]",LINESTRING (760168.6573664969 1823132.71154714...,1028.388216,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.004147e+10,760168.657366,1.823133e+06,1.004147e+10,760939.202088,1.822452e+06
4520,NaN,1096518679,"[10041470241, 10041470243, 10041470244]",LINESTRING (766949.0038125969 1815082.46899628...,665.572341,NaN,<NA>,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.004147e+10,766949.003813,1.815082e+06,1.004147e+10,767306.373604,1.814548e+06


In [42]:
# my_calc_real_lengths

In [43]:
# my_get_tags

In [44]:
data_ways_selected = add_lineID_clone_ways(data_ways_selected, 'VN')

Start adding "LineID" and cloning ways...
   ... 1428 ways doubled, 28 tripled, 94 quadrupled.
   ... finished! (2.354 seconds) 



In [45]:
data_ways_selected['systems']

0       2
1       2
2       1
3       2
4       2
       ..
6283    1
6284    1
6285    1
6286    1
6287    1
Name: systems, Length: 6288, dtype: int64

In [46]:
len(data_ways_selected)

6288

In [47]:
export_to_excel(data_ways_selected)

Start exporting data to Excel files... (may take a few seconds)
<class 'pandas.core.series.Series'>
INFO: Exported Stamm_Leitungen to ../outputs/tbl_Lines_VN_thres-100km_2024-11-05_15-13-16.xlsx
INFO: Exported Nodes to ../outputs/tbl_Nodes_VN_thres-100km_2024-11-05_15-13-16.xlsx
... finished!


In [48]:
# import pandas as pd
# import geopandas as gpd
# from shapely.geometry import Point, LineString

# excel_file = '../outputs/tbl_Nodes_VN_thres-100km_2024-11-05_14-11-57.xlsx'
# data = pd.read_excel(excel_file, sheet_name='Sheet1')

# data['geometry'] = data.apply(lambda row: Point(row['lon'], row['lat']), axis=1)

# # 创建 GeoDataFrame
# gdf = gpd.GeoDataFrame(data, geometry='geometry')

# # 保存为 GeoPackage
# gdf.to_file('../outputs/table_nodes.gpkg', layer='table_nodes', driver='GPKG')

# print('Excel file has been saved as GeoPackage successfully!')
